In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import matplotlib.pyplot as plt
import re
import string

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

pd.set_option('display.max_colwidth',100)

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
df = pd.read_csv('/content/drive/My Drive/MCM_News_Aggregator/uci-news-aggregator.csv',usecols=['TITLE', 'CATEGORY','PUBLISHER'])
df.head(10)

,TITLE,PUBLISHER,CATEGORY
0,"Fed official says weak data caused by weather, should not slow taper",Los Angeles Times,b
1,Fed's Charles Plosser sees high bar for change in pace of tapering,Livemint,b
2,US open: Stocks fall after Fed official hints at accelerated tapering,IFA Magazine,b
3,"Fed risks falling 'behind the curve', Charles Plosser says",IFA Magazine,b
4,Fed's Plosser: Nasty Weather Has Curbed Job Growth,Moneynews,b
5,Plosser: Fed May Have to Accelerate Tapering Pace,NASDAQ,b
6,Fed's Plosser: Taper pace may be too slow,MarketWatch,b
7,Fed's Plosser expects US unemployment to fall to 6.2% by the end of 2014,FXstreet.com,b
8,US jobs growth last month hit by weather:Fed President Charles Plosser,Economic Times,b
9,ECB unlikely to end sterilisation of SMP purchases - traders,Interactive Investor,b


In [ ]:
#I do aspire here to have balanced classes
num_of_categories = 45639
shuffled = df.reindex(np.random.permutation(df.index))
e = shuffled[shuffled['CATEGORY'] == 'e'][:num_of_categories]
b = shuffled[shuffled['CATEGORY'] == 'b'][:num_of_categories]
t = shuffled[shuffled['CATEGORY'] == 't'][:num_of_categories]
m = shuffled[shuffled['CATEGORY'] == 'm'][:num_of_categories]
concated = pd.concat([e,b,t,m], ignore_index=True)
#Shuffle the dataset
concated = concated.reindex(np.random.permutation(concated.index))

In [ ]:
concated.CATEGORY.value_counts()

e    45639
m    45639
b    45639
t    45639
Name: CATEGORY, dtype: int64

In [ ]:
concated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182556 entries, 147404 to 21991
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   TITLE      182556 non-null  object
 1   PUBLISHER  182555 non-null  object
 2   CATEGORY   182556 non-null  object
dtypes: object(3)
memory usage: 5.6+ MB


In [ ]:
concated['title_publish'] = concated.TITLE + " " + concated.PUBLISHER
concated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182556 entries, 147404 to 21991
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   TITLE          182556 non-null  object
 1   PUBLISHER      182555 non-null  object
 2   CATEGORY       182556 non-null  object
 3   title_publish  182555 non-null  object
dtypes: object(4)
memory usage: 7.0+ MB


In [ ]:
concated.CATEGORY.value_counts()

e    45639
m    45639
b    45639
t    45639
Name: CATEGORY, dtype: int64

In [ ]:
# Drop null
concated= concated.dropna(subset=['title_publish'])

In [ ]:
concated['title_publish'].apply(lambda x: len(x.split(' '))).sum()

2055022

In [ ]:
symbols = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', 
           ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', 
           '#', '*', '+', '\\', '•',  '~', '@', '£', '·', '_', 
           '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', 
           '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', 
           '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', 
           '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', 
           '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', 
           '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', 
           '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', 
           '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', 
           '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', 
           '¹', '≤', '‡', '√', ]

def clean_symbol(text):
    text = str(text)
    for symbol in symbols:
        text = text.replace(symbol, '')
    return text

# remove symbols and punctuations 
concated['title_publish'] = concated['title_publish'].apply(lambda x: clean_symbol(x))

In [ ]:
# Dictionary of short form words and mispellings
short_forms_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                    "'cause": "because", "could've": "could have", "couldn't": "could not", 
                    "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", 
                    "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                    "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                    "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", 
                    "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                    "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                    "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                    "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", 
                    "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", 
                    "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", 
                    "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                    "she'll've": "she will have", "she's": "she is", "should've": "should have", 
                    "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", 
                    "that's": "that is", "there'd": "there would", "there'd've": "there would have", 
                    "there's": "there is", "here's": "here is","they'd": "they would", 
                    "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
                    "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", 
                    "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                    "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", 
                    "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", 
                    "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                    "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have",
                    "y'all're": "you all are","y'all've": "you all have","you'd": "you would", 
                    "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", 
                    "you're": "you are", "you've": "you have"}

def clean_shortforms(text):
    clean_text = text
    for shortform in short_forms_dict.keys():
        if re.search(shortform, text):
            clean_text = re.sub(shortform, short_forms_dict[shortform], text)
    return clean_text

# fix short forms
concated['title_publish'] = concated['title_publish'].apply(lambda x: clean_shortforms(x))

In [ ]:
concated['title_publish']  = concated['title_publish'] .str.replace('[^\w\s]','')# unpunctuate

In [ ]:
parser = English()

BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
def clean_text(text):
    text = re.sub("[0-9]+", "",text)
    text = re.sub(' +',' ', text)  
    text = text.lower() # lowercase text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    tokens = parser(text)
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    tokens=  " ".join([word.lower() for word in tokens if word not in string.punctuation and word not in STOP_WORDS])
    return tokens

concated['clean_title_publish'] = concated['title_publish'].apply(lambda x:clean_text(x))

In [ ]:
concated.head(10)

,TITLE,PUBLISHER,CATEGORY,title_publish,clean_title_publish
147404,Marriage is Good for the Heart. Here's Why,The Epoch Times,m,Marriage is Good for the Heart Heres Why The Epoch Times,marriage good heart heres epoch times
168816,"General Motors CEO Mary Barra and former US Attorney Anton Valukas, who ...",KSHB,m,General Motors CEO Mary Barra and former US Attorney Anton Valukas who KSHB,general motors ceo mary barra attorney anton valukas kshb
83459,Banks brace for US sanctions violations probes after BNP Paribas mega ...,euronews,b,Banks brace for US sanctions violations probes after BNP Paribas mega euronews,banks brace sanctions violations probes bnp paribas mega euronews
175861,Got migraines? The FDA has a headband for you,CNBC.com,m,Got migraines The FDA has a headband for you CNBCcom,got migraines fda headband cnbccom
10137,Eric Hill's death: Tears from Andi Dorfman fall as cameras roll,Examiner.com,e,Eric Hills death Tears from Andi Dorfman fall as cameras roll Examinercom,eric hills death tears andi dorfman fall cameras roll examinercom
164488,Review raises questions about Tamiflu,Sky News Australia,m,Review raises questions about Tamiflu Sky News Australia,review raises questions tamiflu sky news australia
86889,Is America Ready To Like Walmart?,Forbes,b,Is America Ready To Like Walmart Forbes,america ready like walmart forbes
63533,US Fed Reserve expects inflation to rise,Sky News Australia,b,US Fed Reserve expects inflation to rise Sky News Australia,fed reserve expects inflation rise sky news australia
8601,"Jack White Shares New Single ""Lazaretto"", Releases ""World's Fastest Released ...",Pitchfork Media,e,Jack White Shares New Single Lazaretto Releases Worlds Fastest Released Pitchfork Media,jack white shares new single lazaretto releases worlds fastest released pitchfork media
164618,Detroit suburb to ticket teens and young adults caught swearing downtown,Daily Mail,m,Detroit suburb to ticket teens and young adults caught swearing downtown Daily Mail,detroit suburb ticket teens young adults caught swearing downtown daily mail


In [ ]:
X_temp=concated['clean_title_publish'].values
y=concated['CATEGORY'].values

In [ ]:
def sent_tokenize(sent):
    temp=nlp(sent)
    return [str(token) for token in temp if not token.is_stop]

In [ ]:
%%time
X=[sent_tokenize(sent) for sent in X_temp]

CPU times: user 26min 17s, sys: 501 ms, total: 26min 17s
Wall time: 26min 19s


In [ ]:
%%time
#from gensim.models import Word2Vec
wv = gensim.models.KeyedVectors.load_word2vec_format("/content/drive/My Drive/MCM_News_Aggregator/Word2Vec/GoogleNews-vectors-negative300.bin", binary=True)
wv.init_sims(replace=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CPU times: user 1min 6s, sys: 6.53 s, total: 1min 13s
Wall time: 1min 40s


In [ ]:
#printing similarity index
print(wv.most_similar('Ireland'))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Irish', 0.7246503829956055), ('Irelands', 0.7044872045516968), ('Cork', 0.7023001909255981), ('Dublin', 0.702103853225708), ('Northern_Ireland', 0.6717302799224854), ('Limerick', 0.6521500945091248), ('Donegal', 0.6396427750587463), ('Galway', 0.6395295858383179), ('Inc._OTCBB_IRLD', 0.6348183751106262), ('Kilkenny', 0.6288274526596069)]


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,shuffle=True)

In [ ]:
def get_embedding(word):
    try:
        embedding=wv[word]
    except:
        embedding=np.zeros((300,))
    return embedding

In [ ]:
X_train_word_average=[np.mean(np.array(list(map(get_embedding,tok_sent))),axis=0) for tok_sent in X_train]
X_test_word_average=[np.mean(np.array(list(map(get_embedding,tok_sent))),axis=0) for tok_sent in X_test]

Logistic Regression

In [ ]:
%%time
logreg = LogisticRegression(n_jobs=-1,solver='newton-cg',penalty='l2',multi_class='multinomial',class_weight=None)
logreg.fit(X_train_word_average, y_train)
y_pred_test_log = logreg.predict(X_test_word_average)
y_pred_train_log = logreg.predict(X_train_word_average)

CPU times: user 904 ms, sys: 632 ms, total: 1.54 s
Wall time: 50.6 s


In [ ]:
d=accuracy_score(y_pred_test_log,y_test)*100
print ('Test Accuracy',d)

e=accuracy_score(y_pred_train_log,y_train)*100
print ('Train Accuracy',e)

Test Accuracy 83.39328427702813
Train Accuracy 83.35211443954049


SGD

In [ ]:
%%time
sgd_search = SGDClassifier(n_jobs=-1, max_iter=15,alpha=0.00001)
sgd_search.fit(X_train_word_average,y_train)

y_pred_test_sgd = sgd_search.predict(X_test_word_average)
y_pred_train_sgd = sgd_search.predict(X_train_word_average)


CPU times: user 9.12 s, sys: 112 ms, total: 9.23 s
Wall time: 5.14 s


In [ ]:
f=accuracy_score(y_pred_test_sgd,y_test)*100
print ('Test Accuracy',f)

g=accuracy_score(y_pred_train_sgd,y_train)*100
print ('Train Accuracy',g)

Test Accuracy 83.47545054503624
Train Accuracy 83.36620026919586


In [ ]:
print(classification_report(y_test, y_pred_test_sgd))

              precision    recall  f1-score   support

           b       0.81      0.78      0.80     13523
           e       0.85      0.88      0.87     13846
           m       0.86      0.88      0.87     13793
           t       0.81      0.79      0.80     13605

    accuracy                           0.83     54767
   macro avg       0.83      0.83      0.83     54767
weighted avg       0.83      0.83      0.83     54767



In [ ]:
# title-category function
def title_to_category1(title):
    title=clean_text(title)
    print(title)

    title=clean_symbol(title)
    print(title)

    title=clean_shortforms(title)
    print(title)

    title=sent_tokenize(title)
    print(title)


    categories = {
                  'e' : 'entertainment',
                  'b' : 'business',  
                  't' : 'science and technology', 
                  'm' : 'health'}
    a=[np.mean(np.array(list(map(get_embedding,tok_sent))),axis=0) for tok_sent in title]
    pridicter1 = sgd_search.predict(a)
    print(pridicter1)
    return categories[max(pridicter1)]

In [ ]:
title_to_category1('Total of 6,666 abortions carried out under new legislation last year')

total abortions carried new legislation year
total abortions carried new legislation year
total abortions carried new legislation year
['total', 'abortions', 'carried', 'new', 'legislation', 'year']
['t' 't' 'e' 't' 't' 'm']


'science and technology'